In [7]:
%cd /home/pasongsongmani/binfo1-datapack1

/home/pasongsongmani/binfo1-datapack1


In [4]:
import pandas as pd
pileup_control_chunk = pd.read_csv('RNA-control-more-filtered.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'], chunksize = 20000)
pileup_control_chunk = list(pileup_control_chunk)

In [5]:
pileup_control = pd.concat(pileup_control_chunk)

In [10]:
pileup_control

,chrom,pos,_ref,count,basereads,quals
0,chr1,3221575,N,80,>>>GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,IIIG>BHGGIIGHIHIIIIIHIIIIIIH@IIIIGIHIHIIIGGCII...
1,chr1,3221576,N,80,>>>A$A$AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,III>EBEECIIGHHIIIIIIGGBIIIGI7IHIIIIGIHHEIGDCII...
2,chr1,3223643,N,82,>>>>>>>>>>>>>>>>>>>>>>>>AAAAAAAAAAAAAAAAAAAAAA...,IIIHIHIIIGIIGIHGGHHID@IGGIII8IIIIIIIII8HHIDIII...
3,chr1,3223644,N,82,>>>>>>>>>>>>>>>>>>>>>>>>AAAAAAAAAAAAAAAAAAAAAA...,IIIHIHIIIGIIGIHGGHHID@IGIIII1IIIIGIIII=FHIBIHI...
4,chr1,3223645,N,81,>>>>>>>>>>>>>>>>>>>>>>>>CCCCCCCCCCCCCCCCCCCCCC...,IIIHIHIIIGIIGIHGGHHID@IGIIIIIIIIIIIII3HHIHIIII...
...,...,...,...,...,...,...
29772946,chr10,25997253,N,180,>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>...,GIIGI?IBI7@F1IIIDFDIIIIEIHIHGIIDGDFHBHGGIGI@IG...
29772947,chr10,25997254,N,180,>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>...,GIIGI?IBI7@F1IIIDFDIIIIEIHIHGIIDGDFHBHGGIGI@IG...
29772948,chr10,25997255,N,180,>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>...,GIIGI?IBI7@F1IIIDFDIIIIEIHIHGIIDGDFHBHGGIGI@IG...
29772949,chr10,25997256,N,180,>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>...,GIIGI?IBI7@F1IIIDFDIIIIEIHIHGIIDGDFHBHGGIGI@IG...


In [2]:
!awk '$4 >= 80 { print $0; }' RNA-control-filtered.pileup > RNA-control-more-filtered.pileup
!tail RNA-control-more-filtered.pileup

awk: cmd. line:1: (FILENAME=RNA-control-filtered.pileup FNR=51598837) fatal: print to "standard output" failed (No space left on device)
chr10	25997248	N	180	>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>	GIIGI?IBI7@F1IIIDFDIIIIEIHIHGIIDGDFHBHGGIGI@IGGGIIIGHIIBIIEIIGDGCIIIGIHHHGIIFHHIIIIIFIECIHIHHIIGII@HEFHIHHIIHHIB<GHBH8IIBIIIIIIHIII@IBHDH9GHIII0GIIFI3IIII=EHGGGBHGBHFII@GDHGIHHGIAI
chr10	25997249	N	180	>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>	GIIGI?IBI7@F1IIIDFDIIIIEIHIHGIIDGDFHBHGGIGI@IGGGIIIGHIIBIIEIIGDGCIIIGIHHHGIIFHHIIIIIFIECIHIHHIIGII@HEFHIHHIIHHIB<GHBH8IIBIIIIIIHIII@IBHDH9GHIII0GIIFI3IIII=EHGGGBHGBHFII@GDHGIHHGIAI
chr10	25997250	N	180	>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [11]:
import re
toremove = re.compile('[<>$*#^!]')
pileup_control['matches'] = pileup_control['basereads'].apply(lambda x: toremove.sub('', x))

In [12]:
pileup_control[['chrom','pos', 'matches']]

,chrom,pos,matches
0,chr1,3221575,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
1,chr1,3221576,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
2,chr1,3223643,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
3,chr1,3223644,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
4,chr1,3223645,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
...,...,...,...
29772946,chr10,25997253,
29772947,chr10,25997254,
29772948,chr10,25997255,
29772949,chr10,25997256,


In [13]:
def shannon_e(base_count, base_length):
  from scipy.stats import entropy 
  val_list=[i/base_length for i in list(base_count.values())]
  return entropy(val_list, base=2)

In [19]:
#from collections import Counter
#entropy = []
#for sequence in pileup_control['matches']:
#    base_length=len(sequence)
#    base_count=Counter(sequence)
#    entropy.append(shannon_e(base_count, base_length))

#pileup_control['entropy']=entropy
pileup_control.to_csv('pre_file_control')
!awk -F ',' 'NR > 1 {print $2, $3, $3+1, $9}' pre_file_control > control-entropy.bedgraph

In [20]:
!head control-entropy.bedgraph

chr1 3221575 3221576 0.173781322679433
chr1 3221576 3221577 0.173781322679433
chr1 3223643 3223644 0.0
chr1 3223644 3223645 0.0
chr1 3223645 3223646 0.0
chr1 3223646 3223647 0.0
chr1 3223647 3223648 0.0
chr1 3223648 3223649 0.0
chr1 3223649 3223650 0.0
chr1 3223650 3223651 0.0


In [22]:
#binding site 앞 -2, 뒤 +3으로 하여 bed file 만들어줌.
#pileup은 origin 1, genome은 0부터 시작이므로 1 빼준다.
!awk -F ',' 'NR > 1 && $9 >=0.8 {print $2"\t"$3-3"\t"$3+3}'  pre_file_control > get_fasta_control.bed
!head get_fasta_control.bed

chr1	4805828	4805834
chr1	4828373	4828379
chr1	4828408	4828414
chr1	4828689	4828695
chr1	4828699	4828705
chr1	4831278	4831284
chr1	4832445	4832451
chr1	4832541	4832547
chr1	4833183	4833189
chr1	4833267	4833273


In [23]:
!bedtools getfasta -fi mm9.fa -bed get_fasta_control.bed -tab > control_RNA_binding_site.csv

Feature (chr4:155643415-155643421) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155690063-155690069) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155690098-155690104) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155692178-155692184) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155694340-155694346) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155694375-155694381) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155694388-155694394) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155694390-155694396) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155694391-155694397) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155694716-155694722) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155694717-155694723) beyond the length of chr4 size (155

Feature (chr4:155707405-155707411) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155707427-155707433) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155707440-155707446) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155707503-155707509) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155707518-155707524) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155707698-155707704) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155708320-155708326) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155709340-155709346) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155709347-155709353) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155709352-155709358) beyond the length of chr4 size (155630120 bp).  Skipping.
Feature (chr4:155709640-155709646) beyond the length of chr4 size (155

Feature (chr9:124079522-124079528) beyond the length of chr9 size (124076172 bp).  Skipping.


In [24]:
!head control_RNA_binding_site.csv

chr1:4805828-4805834	AACTTG
chr1:4828373-4828379	aactac
chr1:4828408-4828414	gtgtaa
chr1:4828689-4828695	GGAAAG
chr1:4828699-4828705	CAGGAC
chr1:4831278-4831284	ATAAAA
chr1:4832445-4832451	cccaca
chr1:4832541-4832547	aatttc
chr1:4833183-4833189	ggagtg
chr1:4833267-4833273	CAAagc
